关联算法中最著名的就是Apriori算法。三个基本概念：支持度、置信度、频繁k项集。

支持度，P（A∩B），既有A又有B的概率，它表现的是A和B两个事件相对整个数据集合同时发生的频繁程度，比如尿布和啤酒的支持度是0.2，表明有20%的消费清单中，消费者同时购买了尿布和啤酒。

置信度，P（B|A），在A发生的事件中同时发生B的概率P（AB）/P（A），它表现的是在AB两个事件的相关程度，和整个数据集合的大小没有关系，比如尿布和啤酒的置信度为0.8，表明在同时购买了两者的消费者中，购买尿布的80%又购买了啤酒。

满足最小支持度阈值的事件称为频繁k项集。设定满足估计阈值下的频繁集才有可能存在关联。

Apriori算法就是挖掘同时满足最小支持度阈值和最小置信度阈值的关联规则。Apriori算法使用频繁项集的先验知识，使用一种称为“逐层搜索”的迭代方法，k项集用于探索（k+1）项集。首先，通过扫描事务（交易）记录，找出所有的频繁1项集，该集合记做L1，然后利用L1找频繁2项集的集合L2，L2找L3，如此下去，直到不能再找到任何频繁k
项集。最后再在所有的频繁集中找出强规则，即产生用户感兴趣的关联规则。其中，Apriori算法具有这样一条性质：任一频繁项集的所有非空子集也必须是频繁的。因为假如P（I）<最小支持度阈值，当有元素A添加到I中时，结果项集（A∩I）不可能比I出现次数更多，因此A∩I也不是频繁的。

Aprior算法步骤：

1. 这里就拿物品交易举例。首先选择交易物品只有一个的集合进行频繁集探测，如果得到的得到的每个物品对应的支持度小于预先设定的概率p，则去掉该项，保留大于P的集合，继而得到所有只有一个物品的集合
2. 继续有两个物品的集合探测，保留支持度大于p的集合，继续三个物品，四个物品...
3. 直到没有新的物品集合加入为止，也就得到所有的频繁集
4. 得到频繁集后，我们再根据设定的强关联规则，得到需要的关联规则集。

Aprior算法的缺点：

1. 物品越多，意味着可选的候选集，也就越多
2. 因每次探测频繁集都需要重新扫描一次数据库，很花时间

Aprior算法是在使用暴力搜索之上的优化，基于若X不是频繁集，即包含X的集合也肯定不是频繁集，如果包含X的是频繁集集，则X也应该是频繁集。

FP-Growth算法的优点：

1. 只要扫描两遍数据库，一次是建立从高到底的支持度顺序；二次是扫描是建立前缀树；
2. 花费的时间明显要小于aprori算法

FP-Growth算法的缺点：

1. 可能生成的构造数会很大，每个购物单只有一个物品；
2. 生成树的大小要小于所有物品数
3. 树的深度小于等于最长物品集
4. 需事先离线生成候选集，而且不能中间停止

在安全领域，Apriori的应用非常广泛，凡是需要挖掘潜在关联关系的都可以尝试使用，比如关联WAF的accesslog与后端数据库的sqllog，识别SSH操作日志中异常操作等。

我们目标是分析出潜在的关联关系，然后作为SVM、KNN等分类算法的特征提取依据之一（比如只提取存在强关联的特征）。机器没有办法直接识别日志，需要逐行将日志文本向量化，最简单的方式就是按照一定的分割符切割成单词向量。

FP-growth算法基于Apriori构建，但采用了高级的数据结构减少扫描次数，大大加快了算法速度。FP-growth算法只需要对数据库进行两次扫描，而Apriori算法对于每个潜在的频繁项集都会扫描数据集判定给定模式是否频繁，因此FP-growth算法的速度要比Apriori算法快。

FP-growth算法发现频繁项集的基本过程如下：

* 构建FP树；
* 从FP树中挖掘频繁项集。

FP-growth算法将数据存储在一种称为FP树的紧凑数据结构中。FP代表频繁模式。一棵FP树看上去与计算机科学中的其他树结构类似，但是它通过链接来连接相似元素，被连起来的元素项可以看成一个链表。

与搜索树不同的是，一个元素项可以在一棵FP树种出现多次。FP树会存储项集的出现频率，而每个项集会以路径的方式存储在数中。存在相似元素的集合会共享树的一部分。只有当集合之间完全不同时，树才会分叉。树节点上给出集合中的单个元素及其在序列中的出现次数，路径会给出该序列的出现次数。相似项之间的链接称为节点链接，用于快速发现相似项的位置。

在互联网环境下存在大量的僵尸主机的扫描行为，僵尸主机频繁更换IP，很难通过IP确定僵尸主机。通过使用FP-growth算法，分析防火墙的拦截日志，挖掘出浏览器的user-agent字段和被攻击的目标URL之间的关联关系，初步确定潜在的僵尸主机

In [9]:
from pyfpgrowth import find_frequent_patterns, generate_association_rules

In [10]:
transactions = [[1, 2, 5],[2, 4],[2, 3],[1, 2, 4],[1, 3],[2, 3],[1, 3],[1, 2, 3, 5],[1, 2, 3]]

In [11]:
patterns = find_frequent_patterns(transactions, 2)
rules = generate_association_rules(patterns, 0.7)

In [12]:
rules

{(5,): ((1, 2), 1.0),
 (1, 5): ((2,), 1.0),
 (2, 5): ((1,), 1.0),
 (4,): ((2,), 1.0)}

In [13]:
filename = './数据集/KnowledgeGraph/sample7.txt'
f = open(filename, 'r')
transactions = []
for line in f.readlines():
    ip, ua, url = line.replace('\n','').split(',')
    transactions.append([ip, ua, url])

In [14]:
transactions

[['ip=ip1', 'ua=ua1', 'target=url1'],
 ['ip=ip2', 'ua=ua1', 'target=url1'],
 ['ip=ip3', 'ua=ua1', 'target=url1'],
 ['ip=ip1', 'ua=ua1', 'target=url2'],
 ['ip=ip2', 'ua=ua1', 'target=url2'],
 ['ip=ip3', 'ua=ua1', 'target=url2'],
 ['ip=ip4', 'ua=ua2', 'target=url2'],
 ['ip=ip5', 'ua=ua3', 'target=url2'],
 ['ip=ip5', 'ua=ua6', 'target=url6'],
 ['ip=ip6', 'ua=ua3', 'target=url4'],
 ['ip=ip7', 'ua=ua4', 'target=url4'],
 ['ip=ip8', 'ua=ua5', 'target=url5']]

In [15]:
patterns = find_frequent_patterns(transactions, 3)
rules = generate_association_rules(patterns, 0.9)

In [16]:
rules

{('target=url1',): (('ua=ua1',), 1.0)}